In [19]:
import os
import sys
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
stemmer = nltk.PorterStemmer()
from collections import Counter

from bs4 import BeautifulSoup
import re

import warnings
warnings.simplefilter("ignore")

In [20]:
data = pd.DataFrame(columns=['text', 'target'])

In [23]:
for file in os.listdir('Dataset/Sport'):
    with open(os.path.join('Dataset/Sport', file), 'r') as f:
        text = f.read()
    data = data.append({'text': text, 'target': 'Спорт'}, ignore_index=True)

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'Dataset/Sport'

In [24]:
data

,text,target


In [25]:
data = data.sample(frac=1).reset_index(drop=True)

In [26]:
class MessageToWordCounterTransform(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        for text in X:
            text = text.lower()
            text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter([word[:-2] for word in text.split() if len(word) > 3 or word == 'NUMBER'])
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [27]:
class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000, vocabulary_=None):
        self.vocabulary_size = vocabulary_size
        if vocabulary_ is not None:
            with open(vocabulary_, 'rb') as f:
                self.vocabulary_ = pickle.load(f)
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [28]:
pipeline = Pipeline([
    ("email_to_wordcount", MessageToWordCounterTransform()),
    ("wordcount_to_vector", WordCounterToVectorTransformer(vocabulary_='vocabulary.pkl')),
])

In [29]:
X, y = data['text'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

X_train = pipeline.transform(X_train).toarray()
X_test = pipeline.transform(X_test).toarray()

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train.values).reshape(-1, 1)
y_test = encoder.transform(y_test.values).reshape(-1, 1)

X = pipeline.transform(X).toarray()
y = encoder.transform(y)

ValueError: With n_samples=0, test_size=0.33 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [5]:
model = SVC(probability=True)

In [6]:
model.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [7]:
accuracy_score(model.predict(X_test), y_test)

NameError: name 'X_test' is not defined

In [8]:
encoder.classes_

NameError: name 'encoder' is not defined

In [9]:
y_pred = model.predict_proba([X_test[1]])

NameError: name 'X_test' is not defined

In [10]:
for pred, type in zip(y_pred[0], article_types):
    print('{} : {}%'.format(type, round(pred * 100, 2)))

NameError: name 'y_pred' is not defined

In [11]:
data.iloc[0]['text'].split('.')[0]

IndexError: single positional indexer is out-of-bounds

In [12]:
data.iloc[0].text

IndexError: single positional indexer is out-of-bounds

In [13]:
text = 'Красноярские депутаты утвердили увеличение бюджета Фонда обязательного медицинского страхования. Решение об этом было принято 20 декабря на сессии Законодательного Собрания Красноярского края.\nДоходная и расходная часть бюджета территориального фонда обязательного медицинского страхования на 2012 и плановый период 2013-2014 годов увеличены. Сделано это в связи с увеличением трансфертов из бюджета федерального фонда. Так, доходы красноярского фонда увеличиваются на 1 млрд 235,9 млн рублей, расходы возрастают на 1 млрд 185,9 млн рублей, источники внутреннего финансирования дефицита бюджета фонда уменьшаются на 50 млн рублей.\nПолученные дополнительно средства будут потрачены на следующие направления: модернизация здравоохранения Красноярского края в части укрепления материально-технической базы медицинских учреждений, выполнение переданных органам государственной власти края полномочий РФ в сфере обязательного медицинского страхования, реализация национального проекта «Здоровье» в части проведения диспансеризации и осуществление единовременных компенсационных выплат медицинским работникам.\nЗа год на обязательное медстрахование в Красноярском крае планируют потратить 32,5 млрд рублей'

In [14]:
vector = pipeline.transform([text])

NameError: name 'pipeline' is not defined

In [15]:
model.predict_proba(vector.toarray())[0].tolist()

NameError: name 'vector' is not defined

In [16]:
vector.toarray().shape

NameError: name 'vector' is not defined

In [17]:
with open('pipeline.pickle', 'wb') as handle:
    pickle.dump(pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'pipeline' is not defined

In [18]:
with open('pipeline.pickle', 'rb') as handle:
    pipeline = pickle.load(handle)

EOFError: Ran out of input